# Limpeza — policy.csv



In [3]:

from pathlib import Path

# (ajuste para o seu usuário/sistema):
CSV_PATH = Path('/Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/policy.csv')

print('CSV_PATH:', CSV_PATH)
print('Existe o arquivo?', CSV_PATH.exists())
if not CSV_PATH.exists():
    raise FileNotFoundError('Arquivo não encontrado. Edite CSV_PATH acima e rode novamente.')


CSV_PATH: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/policy.csv
Existe o arquivo? True


In [4]:
import pandas as pd

df = pd.read_csv(CSV_PATH)
print('Dimensão inicial (linhas, colunas):', df.shape)

df["policy_id"] = df["policy_id"].replace("", pd.NA).fillna("no_data")
df["plan_type"] = df["plan_type"].replace("", pd.NA).fillna("N/A")
df["network_tier"] = df["network_tier"].replace("", pd.NA).fillna("no_data")
df["deductible"] = df["deductible"].fillna(-1)
df["copay"] = df["copay"].fillna(-1)
df["policy_term_years"] = df["policy_term_years"].fillna(-1)
df["policy_changes_last_2yrs"] = df["policy_changes_last_2yrs"].fillna(-1)
df["provider_quality"] = df["provider_quality"].fillna(-1)

cols = list(df.columns)
lower_map = {c.lower(): c for c in cols}
preferred = ['policy_id']

def pick_key():
    for k in preferred:
        if k in lower_map:
            return lower_map[k]
    for lc, orig in lower_map.items():
        if 'policy' in lc and ('id' in lc or 'number' in lc or 'no' in lc):
            return orig
    return cols[0] if cols else None

key_col = pick_key()
if key_col is None:
    raise ValueError('Não foi possível identificar a coluna de chave (policy_id).')
print('Chave utilizada:', key_col)

# Ordenar (mergesort estável): numérico -> string
key_num = pd.to_numeric(df[key_col], errors='coerce')
df = (
    df.assign(_key_num=key_num, _key_str=df[key_col].astype(str))
      .sort_values(by=['_key_num', '_key_str'], kind='mergesort', na_position='last')
      .drop(columns=['_key_num', '_key_str'])
      .reset_index(drop=True)
)

# Remover duplicatas EXATAS
before = len(df)
df = df.drop_duplicates(keep='first')
after = len(df)
print(f'Duplicatas exatas removidas: {before - after} (de {before} → {after})')

# Contradições (mesmo key_col com outros campos diferentes)
other_cols = [c for c in df.columns if c != key_col]
contradiction_ids = []
if other_cols:
    unique_combos = df.drop_duplicates(subset=[key_col] + other_cols)
    counts = unique_combos.groupby(key_col, dropna=False).size()
    contradiction_ids = counts[counts > 1].index.tolist()

# Saídas (baseadas na raiz do repo, um nível acima de 'data')
repo_dir = CSV_PATH.parent.parent
out_clean = repo_dir / 'data' / 'policy_cleaned.csv'
out_contra = repo_dir / 'data' / 'policy_ids_contradicoes.csv'

out_clean.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_clean, index=False)
print('✅ CSV limpo salvo em:', out_clean)

if contradiction_ids:
    pd.DataFrame({key_col: contradiction_ids}).to_csv(out_contra, index=False)
    print(f'⚠️ {len(contradiction_ids)} {key_col}(s) com contradições. Lista salva em:', out_contra)
    for pid in contradiction_ids[:100]:
        print(f'{key_col}::{pid}  erro contraditório')
    if len(contradiction_ids) > 100:
        print(f'... e mais {len(contradiction_ids) - 100}.')
else:
    print('✔ Nenhuma contradição encontrada.')


Dimensão inicial (linhas, colunas): (256, 8)
Chave utilizada: policy_id
Duplicatas exatas removidas: 0 (de 256 → 256)
✅ CSV limpo salvo em: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/policy_cleaned.csv
✔ Nenhuma contradição encontrada.
